In [1]:
import pandas as pd
import os, sys
from sklearn.gaussian_process import GaussianProcessRegressor
import shapefile
from functools import partial
import pyproj
from shapely.geometry import shape, Point, mapping
from shapely.ops import transform
processeddir = "../data/processed/"
rawdir = "../data/raw/"
df = pd.read_csv(os.path.join(processeddir,"nyctaxiclean.csv"), dtype={"store_and_fwd_flag": "object"})

In [2]:
df["pickup_float"] = pd.to_datetime(df["pickup_datetime"]).apply(lambda x: x.timestamp())
df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])
df["dropoff_datetime"] = pd.to_datetime(df["dropoff_datetime"])
df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])
df["dropoff_datetime"] = pd.to_datetime(df["dropoff_datetime"])

In [3]:
#get centroids from shapesfile from NYC taxi

def get_centroids(filename=rawdir+"taxi_zones/taxi_zones"):
    sf = shapefile.Reader(filename)

    project = partial(
        pyproj.transform,
        pyproj.Proj(init='epsg:2263', preserve_units=True), # NAD_1983_StatePlane_New_York_Long_Island_FIPS_3104_Feet
        pyproj.Proj(init='epsg:4326')) # wgs84 (lat/lng) 
 
    centroids = list(map(lambda x: transform(project, shape(x.__geo_interface__)).centroid, sf.shapes()))
    
    df = pd.DataFrame(sf.records())
    df = df.rename(columns={3:"name"})
    df["centroid_x"] = list(map(lambda x: x.x, centroids))
    df["centroid_y"] = list(map(lambda x: x.y, centroids))
    return df

centroid_df = get_centroids()

In [4]:
df["pickup_hour"] = df["pickup_datetime"].apply(lambda x: x.hour)
df["pickup_dayofweek"] = df["pickup_datetime"].apply(lambda x: x.weekday())
df["pickup_dayofmonth"] = df["pickup_datetime"].apply(lambda x: x.day)


In [5]:
centroid_dataset = df.merge(centroid_df, left_on="pickup_neighbourhood", right_on="name")
centroid_dataset

,Unnamed: 0,medallion,hack_license,vendor_id_x,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,...,pickup_dayofweek,pickup_dayofmonth,0,1,2,name,4,5,centroid_x,centroid_y
0,0,91F6EB84975BBC867E32CB113C7C2CD5,AD8751110E6292079EB10EB9481FE1A6,CMT,1,N,2013-04-04 18:47:45,2013-04-04 19:00:25,1,759,...,3,4,141,0.041514,0.000077,Lenox Hill West,141,Manhattan,-73.959635,40.766948
1,9,13A57EE874E2560DF1F9D6C639BB7DAB,5FE3733438871FE98C05D259E8693750,CMT,1,N,2013-04-03 18:05:38,2013-04-03 18:23:06,1,1048,...,2,3,141,0.041514,0.000077,Lenox Hill West,141,Manhattan,-73.959635,40.766948
2,32,D83519B044B750F146A3A349197C80AA,321DDAF4C11879A55D71E254A3CCFFA5,CMT,1,N,2013-04-05 07:16:03,2013-04-05 07:24:41,1,518,...,4,5,141,0.041514,0.000077,Lenox Hill West,141,Manhattan,-73.959635,40.766948
3,72,8FDE065D961FFE6CB6B4CA1F47525C6D,5A81C4DD7805B2F8686398C7D42FC2C7,CMT,1,N,2013-04-03 20:07:05,2013-04-03 20:14:32,1,446,...,2,3,141,0.041514,0.000077,Lenox Hill West,141,Manhattan,-73.959635,40.766948
4,106,34EA29ABFCE6B78138FEA90D330101B1,9C47390E9A6BB2C8C6CBCEB388612AE4,CMT,1,Y,2013-04-03 21:16:31,2013-04-03 21:20:43,1,252,...,2,3,141,0.041514,0.000077,Lenox Hill West,141,Manhattan,-73.959635,40.766948
5,117,30DB18A7927FB0496BF91BEA23D961E9,4384DA84EE515C5A6D2906D3055E89C0,CMT,1,N,2013-04-05 14:49:05,2013-04-05 14:57:01,1,476,...,4,5,141,0.041514,0.000077,Lenox Hill West,141,Manhattan,-73.959635,40.766948
6,151,2D1FD2323BE2C8C086FFE5661888892D,06C11DFE127C0F0ACDE38B2850D5F9B7,CMT,1,N,2013-04-04 12:26:55,2013-04-04 12:34:56,1,480,...,3,4,141,0.041514,0.000077,Lenox Hill West,141,Manhattan,-73.959635,40.766948
7,238,385B40B09DF15C8D150AF46E1B4A3AB4,1B1B09F687E9419E47E5EF4518C53E75,VTS,1,NaN,2013-04-12 18:35:00,2013-04-12 18:38:00,1,180,...,4,12,141,0.041514,0.000077,Lenox Hill West,141,Manhattan,-73.959635,40.766948
8,290,FD9900398CC8CBEF7D3E7890FA854FA2,A7D8E5B338FA2134D13EF1633E376492,VTS,1,NaN,2013-04-12 21:28:00,2013-04-12 21:47:00,6,1140,...,4,12,141,0.041514,0.000077,Lenox Hill West,141,Manhattan,-73.959635,40.766948
9,326,78FD0655D0CBD598C17BFA40654850E2,A5D7A4EA72DAF6D393087ACA90838E46,VTS,1,NaN,2013-04-12 20:15:00,2013-04-12 20:26:00,1,660,...,4,12,141,0.041514,0.000077,Lenox Hill West,141,Manhattan,-73.959635,40.766948


In [114]:
#predict a fare given startpoint and destination

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
# 4 POC neighbourhoods
chosen_neighbourhoods = ["Upper East Side South", "Midtown Center", "Flatiron", "JFK Airport"]

# as this dataset is not aggregated we can use random 4-fold CV
kf = KFold(n_splits=4, shuffle=True)

fare_models = []
preds = []
rmse = []
neigh_df = centroid_dataset[centroid_dataset["pickup_neighbourhood"].isin(chosen_neighbourhoods)][["pickup_hour", "pickup_dayofweek", "dropoff_longitude", "dropoff_latitude", "fare_amount", "tip_amount"]]
for train_index, test_index in kf.split(neigh_df):
    fare_model = RandomForestRegressor()
    fare_models.append(fare_model)
    train_df = neigh_df.iloc[train_index]
    test_df = neigh_df.iloc[test_index]
    x_keys = ["pickup_hour", "pickup_dayofweek", "dropoff_longitude", "dropoff_latitude"]
    y_keys = ["fare_amount", "tip_amount"]
    fare_model.fit(train_df[x_keys], train_df[y_keys])

    preds.append(fare_model.predict(test_df[x_keys]))
    rmse.append((np.sqrt(mean_squared_error(test_df[y_keys[0]],[x[0] for x in preds[-1]])), 
            np.sqrt(mean_squared_error(test_df[y_keys[1]],[x[1] for x in preds[-1]]))))

print(rmse)

[(12.747741367747816, 2.8565813319770026), (12.748133130203438, 2.8748881960663355), (12.734691955499766, 2.8709108283870015), (12.778356007727016, 2.851119346700868)]


In [117]:
np.std([x[1] for x in rmse])

0.009819589554984033

In [120]:
fare_models[-1].feature_importances_

array([0.08336131, 0.06578142, 0.39830618, 0.4525511 ])

In [132]:
#predict a fare given time of day and pickup neighbourhood

chosen_neighbourhoods = ["Upper East Side South", "Midtown Center", "Flatiron", "JFK Airport"]

# as this dataset is not aggregated we can use random 4-fold CV
kf = KFold(n_splits=4, shuffle=True)

fare_models = []
preds = []
rmse = []
neigh_df = centroid_dataset[centroid_dataset["pickup_neighbourhood"].isin(chosen_neighbourhoods)][["pickup_hour", "pickup_dayofweek", "pickup_neighbourhood", "fare_amount", "tip_amount"]]

#OHE the pickup_neighbourhood
for i in range(len(chosen_neighbourhoods)):
    neigh_df[i] = (neigh_df["pickup_neighbourhood"] == chosen_neighbourhoods[i]).astype(int)
    
neigh_df = neigh_df.drop("pickup_neighbourhood", axis=1)

In [134]:
for train_index, test_index in kf.split(neigh_df):
    fare_model = RandomForestRegressor()
    fare_models.append(fare_model)
    train_df = neigh_df.iloc[train_index]
    test_df = neigh_df.iloc[test_index]
    x_keys = ["pickup_hour", "pickup_dayofweek"] + list(range(len(chosen_neighbourhoods)))
    y_keys = ["fare_amount", "tip_amount"]
    fare_model.fit(train_df[x_keys], train_df[y_keys])

    preds.append(fare_model.predict(test_df[x_keys]))
    rmse.append((np.sqrt(mean_squared_error(test_df[y_keys[0]],[x[0] for x in preds[-1]])), 
            np.sqrt(mean_squared_error(test_df[y_keys[1]],[x[1] for x in preds[-1]]))))

print(rmse)

[(8.940778962999726, 2.5948808784499064), (8.792814718379235, 2.581849826891663), (8.760484044953822, 2.568522295371402), (8.784875548938043, 2.579961548953915)]


In [138]:
np.std([x[1] for x in rmse])

0.009351519969197294

what follows is the unsuccessful attempt at training KDE on this data; preserved for reference

In [18]:
from statsmodels.nonparametric.kernel_density import KDEMultivariate
from sklearn.model_selection import KFold
# 4 POC neighbourhoods
chosen_neighbourhoods = ["Upper East Side South", "Midtown Center", "Flatiron", "JFK Airport"]

# as this dataset is not aggregated we can use random 4-fold CV
kf = KFold(n_splits=4, shuffle=True)

kde_models = []
likelihood = []
for neigh in chosen_neighbourhoods:
    neigh_df = centroid_dataset[centroid_dataset["pickup_neighbourhood"] == neigh][["pickup_hour", "pickup_dayofweek", "dropoff_longitude", "dropoff_latitude", "fare_amount", "tip_amount"]]
    for train_index, test_index in kf.split(neigh_df):
        train_df = neigh_df.iloc[train_index]
        test_df = neigh_df.iloc[test_index]
        kde_models.append(KDEMultivariate(train_df, var_type='oocccc'))
        likelihood.append(kde_models[-1].pdf(test_df))

In [20]:
import numpy as np

kde_models = []
centroid_dataset["fare_tip"] = centroid_dataset["fare_amount"] + centroid_dataset["tip_amount"]
for neigh in chosen_neighbourhoods:
    neigh_df = centroid_dataset[centroid_dataset["pickup_neighbourhood"] == neigh][["pickup_hour", "pickup_dayofweek", "dropoff_longitude", "dropoff_latitude", "fare_tip"]]
    kde_models.append(KDEMultivariate(neigh_df, var_type='ooccc'))



In [55]:
# also estimate probability of given event
# P(y|x) = P(y,x)/P(x)
x_models = []
for neigh in chosen_neighbourhoods:
    neigh_df = centroid_dataset[centroid_dataset["pickup_neighbourhood"] == neigh][["pickup_hour", "pickup_dayofweek", "dropoff_longitude", "dropoff_latitude"]]
    x_models.append(KDEMultivariate(neigh_df, var_type='oocc'))

In [24]:
grouped_df = df.groupby("pickup_neighbourhood")
sig_neigh = grouped_df.count().sort_values("medallion", ascending=False).index[:50]

In [28]:
sig_centroids = centroid_df[centroid_df["name"].isin(sig_neigh)]

In [65]:
len(y_x_data[0])

1

In [100]:
y_x = []
x = []
fares = np.linspace(1,100,25)
probs = []
x_data = [[0, 0, y["centroid_x"], y["centroid_y"]] for i, y in sig_centroids.iterrows()]   
y_x_data = [[y + [j]] for y in x_data for j in fares]

for i, neigh in enumerate(chosen_neighbourhoods):
    x.append(x_models[i].cdf(x_data[i]))
    y_x.append(kde_models[i].cdf(y_x_data)) # cdf probability that it's less than this fare
    probs.append(np.exp(y_x[-1] - x[-1])) # baye's theorem... np.exp because we have log likelihood
len(probs)

4

In [102]:
probs[0]

array([1.0003423 , 0.98137904, 0.93529751, ..., 0.85124727, 0.85124068,
       0.85123168])

In [81]:
len(fares)

25

In [95]:
centroid_df[centroid_df["name"].isin(chosen_neighbourhoods)]


,0,1,2,name,4,5,centroid_x,centroid_y
89,90,0.030759,0.000055,Flatiron,90,Manhattan,-73.996971,40.742279
131,132,0.245479,0.002038,JFK Airport,132,Queens,-73.786533,40.646985
160,161,0.035804,0.000072,Midtown Center,161,Manhattan,-73.977698,40.758028
236,237,0.042213,0.000096,Upper East Side South,237,Manhattan,-73.965635,40.768615


In [50]:
fares

array([  1.   ,   5.125,   9.25 ,  13.375,  17.5  ,  21.625,  25.75 ,
        29.875,  34.   ,  38.125,  42.25 ,  46.375,  50.5  ,  54.625,
        58.75 ,  62.875,  67.   ,  71.125,  75.25 ,  79.375,  83.5  ,
        87.625,  91.75 ,  95.875, 100.   ])